In [227]:
# load packages
import numpy as np
from IPython.display import SVG
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline
from pathlib import Path
import pandas as pd
import pickle
import glob

repo = Path('../')
datadir = Path('../raw_processed_data')



df.read_pickle(datadir.joinpath('raw_processed_validated.pkl'))

### Create input data matrix

shape of $data$: (m,n_channel,n_adj*2+1,37)


In [362]:
def create_data_matrix(df,n_adj=1,include_std=True):
    lm_std_cols = [c for c in df.columns if c.startswith('DBDT_STD_Ch1')]
    lm_cols = [c for c in df.columns if c.startswith('DBDT_Ch1')]
    hm_std_cols = [c for c in df.columns if c.startswith('DBDT_STD_Ch2')]
    hm_cols = [c for c in df.columns if c.startswith('DBDT_Ch2')]
    all_rows = np.arange(n_adj*2,len(df)-n_adj*2,2)
    m = len(all_rows)
    if include_std:
        n_channel=4
    else:
        n_channel=2
    data = np.zeros((m,n_channel,n_adj*2+1,37),dtype=float)

    for i,row in enumerate(all_rows):
        if i%5000==0:
            print('iteration {} of {}'.format(i,m))
        #get the correct soundings
        df_tmp = df.loc[row-n_adj*2:row+n_adj*2+1]
        HMind = df_tmp.index[df_tmp.CHANNEL_NO==2]
        LMind = HMind+1

        #Create data matrix
        lm_data_zero_padded = np.c_[df_tmp.loc[LMind,lm_cols].values,
                                    9999.0*np.ones((n_adj*2+1,9))]
        hm_data = df_tmp.loc[HMind,hm_cols].values


        if include_std:
            lm_std_zero_padded = np.c_[df_tmp.loc[LMind,lm_std_cols].values,
                                        9999.0*np.ones((n_adj*2+1,9))]
            hm_std = df_tmp.loc[HMind,hm_std_cols].values
            data[i] = np.stack((hm_data,lm_data_zero_padded,hm_std,lm_std_zero_padded))
        else:
            data[i] = np.stack((hm_data,lm_data_zero_padded))
    return data



### Create and save a data matrix for 1-4 adjacent soundings

In [ ]:
include_std=True
for n_adj in np.arange(1,5):
    print('creating data for n_adj={}'.format(n_adj))
    data = create_data_matrix(df,n_adj=n_adj,include_std=include_std)
    np.save(datadir.joinpath('data_nadj{}_std{}.npy'.format(n_adj,include_std)),data)